In [25]:
import re
from sys import prefix
import pandas as pd
import random
import os
import shutil
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt


class sudoku:
    def __init__(self, init=False, path='benchmarks/INST_36x36', solver='ACS', execl='results/charts/base.xlsx'):
        self.path = path
        self.opt_path = path + '/100'
        self.solver = solver
        self.data_path = 'results/' + solver
        # self.records = pd.read_excel(execl, header=[0], index_col = [0])
        if init:
            self.records = self.records.drop(index=self.records.index)
            self.get_info(path)
            
    def get_benchmarks(self, percentage=0):
        file_list = os.listdir(self.opt_path)
        target_path = self.path + '/' + str(percentage)
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        for file in tqdm(file_list, desc=self.data_path):
            path = os.path.join(self.opt_path, file)
            with open(path) as f:
                grid = []
                order = f.readline()[:-1]
                is_sat = f.readline()
                square = int(order)*int(order)
                for i in range(square):
                    grid.append(f.readline().split())
                index = 0
                total = (100-percentage)*square*square/100
                while index < total:
                    x = random.randint(0, square-1)
                    y = random.randint(0, square-1)
                    if grid[x][y] != '-1':
                        grid[x][y] = '-1'
                        index += 1
                filename = file.split('_')
                filename[1] = str(percentage)
                filename = '_'.join(filename)
                with open(target_path + '/' + filename, 'w') as out:
                    out.write(order + '\n')
                    out.write(is_sat)
                    for i in range(square):
                        line = ''
                        for j in range(square):
                            line = line + grid[i][j] + '\t'
                        out.write(line[:-1] + '\n')

    def get_info(self, path):
        folder_list = os.listdir(path)
        for folder in folder_list:
            new_path = os.path.join(path, folder)
            if os.path.isdir(new_path):
                self.get_info(new_path)
            else:
                filename = new_path.split('/')[-1]
                category = filename.split['_'][1]
                self.records.loc[len(self.records.index)] = [filename, category]
        
    def save_to(self, filename):
        self.records.to_excel(filename)
        
    def arrange(self):
        # for filename in tqdm(folder_list, desc=folder_path):
        file_list = os.listdir(self.data_path)
        for file in tqdm(file_list, desc=self.data_path):
            path = os.path.join(self.data_path, file)
            with open(path) as f:
                try:
                    if self.solver == 'ACS':
                        while f.readline():
                            result = f.readline()[:-1]
                            if result == '0':
                                result = 'sat'
                            elif result == '1':
                                result = 'timeout'
                            f.readline()
                            info = f.readline()
                            time = info.split(' : ')[-1][:-5]
                            name = info.split(' : ')[0].split('/')[-1]
                            # print(name)
                            self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, int(time)]
                    elif self.solver == 'SAT':
                        while f.readline():
                            result = 'error'
                            info = f.readline()
                            while info[:6] != 'c proc':
                                info = f.readline()
                            status = f.readline()
                            if len(status) >= 10:
                                result = 'timeout'
                            elif len(status) <= 2:
                                result = 'sat'
                            # times = float(re.split('[sm]', info)[-3])
                            info = f.readline()
                            while(info and info[:3] != '../'):
                                info = f.readline()
                            time = info.split(' : ')[-1][:-5]
                            name = info.split(' : ')[0].split('/')[-1].split('.')[0] + '.txt'
                            # print(name)
                            self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, time]
                except:
                    print(path)
    

In [26]:
a = sudoku()
for i in range(0, 100, 5):
    a.get_benchmarks(percentage=i)

results/CP: 100%|██████████| 100/100 [00:00<00:00, 1984.63it/s]
